In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!


#from huggingface_hub import notebook_login

#notebook_login()

In [2]:
import json
import pandas as pd
from copy import deepcopy
import random
from tqdm.notebook import tqdm

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AdamW
)
import pickle

from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [3]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [4]:
def split_input_dict(input_dict, ratio = 0.01, seed = 42):
    split_point = int(len(input_dict['utterance']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['utterance'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['utterance'] = input_dict['utterance'][:split_point]
    train_dict['utterance'] = input_dict['utterance'][split_point:]
    return train_dict, valid_dict

In [5]:
def read_input(path):
    data = pd.read_csv(path)
    input_dict = {"utterance":[]}
    for _, row in tqdm(data.iterrows()):
        for q_col in ['질문_1', '질문_2']:
            for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
                input_dict["utterance"].append({"question":row[q_col],"answer":row[a_col]})
            
            
    train_dict,valid_dict = split_input_dict(input_dict)
    train_question = []
    train_answer = []
    for pairs in tqdm(train_dict['utterance']):
        train_question.append(pairs['question'])
        train_answer.append(pairs['answer'])

    valid_question = []
    valid_answer = []
    for pairs in tqdm(valid_dict['utterance']):
        valid_question.append(pairs['question'])
        valid_answer.append(pairs['answer'])

    
    return train_question ,train_answer , valid_question , valid_answer

In [6]:
train_q ,train_a , valid_q , valid_a = read_input("./bigdata/train.csv")

0it [00:00, ?it/s]

  0%|          | 0/6376 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [23]:
####데이터 3개씩 끊어보는 기능
n=2
for ind,i in enumerate(zip(train_q,train_a)):
    if "도배지" in i[0]:
        print(i[0]+" ## "+i[1])
        print()
    if ind<30 and False:
        print(i[0]+" ## "+i[1])
        print()

도배를 마친 후에 남은 도배지 처리 방법은 무엇인가요? ## 도배지를 처리하는 가장 일반적인 방법은 폐기물 스티커를 부착하는 것입니다. 이러한 스티커는 동사무소나 편의점, 일반 슈퍼 등에서 구매할 수 있습니다. 또한, 아파트의 경우에는 경비실에 비치되어 있을 수도 있으니 해당 사항을 확인하시고 문의하시는 것이 좋습니다. 이를 통해 도배지를 적절히 처리할 수 있습니다.

도배지 꼬임에 대해 설명해줘 ## 도배지 꼬임은 도배 시에 주름 모양의 형태가 발생하는 현상입니다. 이러한 도배지 꼬임이 발생하는 원인과 책임 소재 그리고 해결 방법에 대해 설명드리겠습니다.  1. 공간 내 높은 습도 원인: 습기가 도배지에 스며들거나 습기로 인해 도배지가 팽창할 때 꼬임이 발생할 수 있습니다. 책임 소재: 건물의 소유자나 거주자가 습기 관리에 책임이 있습니다. 해결 방법: 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업해야 합니다.  2. 벽면의 손상(불균형) 원인: 벽면에 손상이 있을 경우 손상된 벽을 통해 습기가 유입하는 등의 원인으로 인해 꼬임이 발생할 가능성이 있습니다. 책임 소재: 벽면에 손상을 방생시킨 주체가 책임을 집니다. 해결 방법: 벽면을 제대로 복원하고 평평하게 균형을 맞춘 후 재작업 해야 합니다. 해당 작업은 개인이 하기 어려우니 전문가의 도움을 받는 것을 추천합니다.  3. 불안정한 온도 원인: 온도의 큰 변화는 도배지의 팽창과 수축을 유발하고 접착제의 안정성에도 영향을 미쳐 도배지 꼬임이 발생할 가능성이 있습니다. 책임 소재: 거주자가 온도 관리에 책임이 있습니다. 해결 방법: 적정 온도를 유지하기 위해 냉·난방을 활용하고, 꼬임이 사라지지 않는다면 전문가의 도움을 받아 부분 혹은 전체 재작업을 해야 합니다.  이러한 원인과 책임 소재에 맞춰서 해결 방법을 통해 도배지 꼬임을 최소화하고 원활한 시공을 할 수 있기를 바랍니다.

한 롤의 도배지로 몇평을 할 수 있는지 알고 싶습니다. ## 도배지의 경우 1롤 당 약 10평

In [20]:
length_of_answer = 0
for ind, i in enumerate(zip(train_q,train_a)):
    length_of_answer += len(trg_tokenizer(i[1])['input_ids'])

print(length_of_answer/len(train_a))

65.01333124215809


In [24]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [25]:
test_q= read_input('./bigdata/test.csv')

In [27]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    if(i>30 and i<60):
        print(sr_text)
        print()


몰딩 수정을 예방하기 위해 건물 내부에서 어떤 종류의 환경 관리가 필요한가요?

KMEW 세라믹 지붕재의 단점에 대해 알려주세요. 또한, 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?

줄퍼티 마감은 무엇인가요? 또한, 액체방수공사는 무엇을 하는 것인가요?

페인트 하도재 없이 페인트를 바로 칠할 경우 어떤 문제가 발생할 수 있나요?

바닥재가 남으면 어떻게 처리하는 게 좋을까요? 그리고 장판이 남을 때 어떻게 처리해야 하나요?

도배지에 생긴 반점을 없애기 위해 가장 효과적인 방법은 무엇인가요?

새집증후군의 주요 원인은 무엇인가요?

방청도료 도장 작업을 위해 필요한 단계는 무엇인가요? 또한, 콘크리트 벽에 구멍을 뚫는 방법에는 어떤 도구나 기술을 사용해야 하나요?

어떤 종류의 실내 식물을 선택해야 식물을 효과적으로 가꾸는 데 도움이 될까요? 그리고 인테리어에 가장 많이 사용되는 도배재료는 무엇인가요?

원목마루와 롱브릭타일에 대해 설명해주세요. 각각의 단점과 특징은 무엇인가요?

침실을 더 아늑하게 꾸밀 수 있는 방법에는 어떤 것이 있을까요와 아이가 있는 집을 꾸밀 때 안전을 위해 고려해야 할 요소는 무엇인가요의 관계는 무엇인가요?

인테리어에서 컬러매치를 위한 효과적인 팁이 있을까요? 그리고 복도나 협소한 공간을 확장시키기 위해 가장 효과적인 방법이 무엇일까요?

그라스울 보온판의 주요 장점 중 하나인 흡음 성능은 어떻게 발휘될까요?"

미네랄울 보온판은 왜 고속 회전원심공법으로 제조되는 건가요?"

도배 후 필름 시공은 어떤 경우에 추천하시나요? 또한, 낡은 목재 가구의 흠집을 숨기는 방법을 알려주세요.

입구나 복도의 표면에 사용하기 적합한 페인트 종류는 무엇이며, 이 공간을 환영스럽게 꾸미는 데 있어 어떤 인테리어 요소가 중요한가요?

도배지가 먼지나 연기로 인해 얼마나 빨리 오염될 수 있나요? 그리고 습도가 높을 때 곰팡이가 어떻게 발생하는지 자세히 알고 싶습니다.

방청페인트를 시공하는 방법에는 어떤 단계가 포함되나요? 또한, 배관